In [ ]:
!pip install filterpy
import json
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import leastsq, curve_fit, least_squares
from collections import deque
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
import random
import matplotlib.cm as cm
import matplotlib.colors as mcolors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=f43dc38797a38ac13fa808682c79dc55574912ba41a8454272a13e4f9c14755a
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8fa2d15e1
Successfully built filterpy


In [ ]:
def load_trajectory_data(json_file, start=None, end=None):
    # Load JSON data
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Extract and convert values
    x_vals = [point['x'] / 1000 for point in data]  # Convert mm to m
    y_vals = [point['y'] / 1000 for point in data]
    z_vals = [point['z'] / 1000 for point in data]
    t_vals = [point['t'] / 1_000_000 for point in data]  # Convert µs to s

    # Normalize time so the first timestamp is at t = 0
    t_start = t_vals[0]
    t_vals = [t - t_start for t in t_vals]

    # Apply slicing only if start or end is provided
    return (
        x_vals[start:end] if start is not None or end is not None else x_vals,
        y_vals[start:end] if start is not None or end is not None else y_vals,
        z_vals[start:end] if start is not None or end is not None else z_vals,
        t_vals[start:end] if start is not None or end is not None else t_vals,
    )

x_vals, y_vals, z_vals, t_vals = load_trajectory_data("drop3.json", 0)

FileNotFoundError: [Errno 2] No such file or directory: 'drop3.json'

In [ ]:
import plotly.express as px
import pandas as pd  # Import pandas

def plot_3d_scatter(x_vals, y_vals, z_vals):
    # Create a pandas DataFrame from the data
    df = pd.DataFrame({'x': x_vals, 'y': y_vals, 'z': z_vals, 'index': list(range(len(x_vals)))})

    # Create a 3D scatter plot using the DataFrame
    fig = px.scatter_3d(
        df,  # Pass the DataFrame
        x='x', y='y', z='z',
        labels={'x': 'x (meters)', 'y': 'y (meters)', 'z': 'z (meters)'},
        title="3D Coordinate Plot",
        hover_data={'x': False, 'y': False, 'z': False, 'index': True}  # Use column names
    )

    # Add the index as text labels for each point
    fig.update_traces(
        text=[f"Idx: {i}" for i in range(len(x_vals))],  # Add index as text labels
        marker=dict(
            size=10,  # Circle size
            color='rgba(0, 0, 0, 0)',  # Transparent fill
            line=dict(color='blue', width=2)  # Outline color and width
        )
    )

    # Set axis ranges dynamically
    range_x = [min(x_vals), max(x_vals)]
    range_y = [min(y_vals), max(y_vals)]
    range_z = [min(z_vals), max(z_vals)]

    # Update the layout to set the y-axis as the vertical axis
    fig.update_layout(
        scene=dict(
            xaxis=dict(title='x (meters)', range=range_x),
            yaxis=dict(title='y (meters)', range=range_y),
            zaxis=dict(title='z (meters)', range=range_z),
            aspectmode="data",
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Show the plot
    fig.show()
    return fig

def add_point(fig, x, y, z):
    # Add a single red point to the figure
    fig.add_scatter3d(
        x=[x], y=[y], z=[z],
        mode='markers',
        marker=dict(size=10, color='red'),
        name='Interception Point'
    )

    fig.show()
    return fig

In [ ]:
def position_model(t, g, v0, r0):
    return 0.5 * g * t**2 + v0 * t + r0

def fit_gravity_component(t_vals, pos_vals):
    # Assuming this function fits the gravity model to each axis (x, y, or z)
    try:
        # Fit the model and return the gravity component, velocity, and position
        popt, _ = curve_fit(lambda t, g, v0, r0: position_model(t, g, v0, r0), t_vals, pos_vals, p0=[-9.81, 0, 0])
        return popt  # Returns g_x, v_x0, r_x0
    except Exception as e:
        print(f"Error fitting gravity component: {e}")
        return None  # If fitting fails, return None

def estimate_gravity(x_vals, y_vals, z_vals, t_vals):
    # Fit gravity components for each axis
    print("Fitting gravity for x-axis...")
    result_x = fit_gravity_component(t_vals, x_vals)
    print("Fitting gravity for y-axis...")
    result_y = fit_gravity_component(t_vals, y_vals)
    print("Fitting gravity for z-axis...")
    result_z = fit_gravity_component(t_vals, z_vals)

    # Check if any of the results is None (fitting failed)
    if result_x is None or result_y is None or result_z is None:
        print("Error: Fitting failed for one or more axes.")
        return None, None

    g_x, v_x0, r_x0 = result_x
    g_y, v_y0, r_y0 = result_y
    g_z, v_z0, r_z0 = result_z

    # Gravity vector and magnitude
    gravity_vector = np.array([g_x, g_y, g_z])
    gravity_magnitude = np.linalg.norm(gravity_vector)  # Magnitude of the vector

    return gravity_vector, gravity_magnitude

def average_gravity(data):
    gravity_vectors = []
    gravity_magnitudes = []

    # Loop over each dataset and estimate gravity
    for file, (start, end) in data.items():
        print(f"Processing {file}...")
        x_vals, y_vals, z_vals, t_vals = load_trajectory_data(file, start, end)

        if len(x_vals) == 0 or len(y_vals) == 0 or len(z_vals) == 0 or len(t_vals) == 0:
            print(f"Warning: Data for {file} is empty or malformed.")
            continue

        gravity_vector, gravity_magnitude = estimate_gravity(x_vals, y_vals, z_vals, t_vals)

        if gravity_vector is None or gravity_magnitude is None:
            print(f"Error: Gravity estimation failed for {file}. Skipping.")
            continue

        gravity_vectors.append(gravity_vector)
        gravity_magnitudes.append(gravity_magnitude)

    if len(gravity_vectors) == 0:
        print("Error: No valid gravity vectors computed.")
        return None, None

    # Convert list of gravity vectors into a numpy array
    gravity_vectors = np.array(gravity_vectors)

    # Calculate average gravity vector
    gravity_vector = np.mean(gravity_vectors, axis=0)
    current_magnitude = np.linalg.norm(gravity_vector)
    scaling_factor = 11.1 / current_magnitude

    scaled_gravity_vector = scaling_factor * gravity_vector
    avg_gravity_magnitude_from_vector = np.linalg.norm(scaled_gravity_vector)

    # Calculate average gravity magnitude
    avg_gravity_magnitude = np.mean(gravity_magnitudes)

    print(f"Average gravity vector: {gravity_vector}")

    print(f"Rescaled gravity vector: {scaled_gravity_vector}")

    return scaled_gravity_vector

# Example dataset with file names and start/end indices
data = {
    "drop1.json": [0, 22],
    "drop2.json": [1, 18],
    "drop3.json": [5, 15],
    "drop4.json": [None, None]
}

# Call the average gravity function
gravity_vector = average_gravity(data)


Processing drop1.json...
Fitting gravity for x-axis...
Fitting gravity for y-axis...
Fitting gravity for z-axis...
Processing drop2.json...
Fitting gravity for x-axis...
Fitting gravity for y-axis...
Fitting gravity for z-axis...
Processing drop3.json...
Fitting gravity for x-axis...
Fitting gravity for y-axis...
Fitting gravity for z-axis...
Processing drop4.json...
Fitting gravity for x-axis...
Fitting gravity for y-axis...
Fitting gravity for z-axis...
Average gravity vector: [-0.50884062  9.58176992  1.21602112]
Rescaled gravity vector: [-0.5839661  10.99642713  1.39555507]


In [ ]:
def estimate_trajectory(x_vals, y_vals, z_vals, t_vals, gravity_vector):
    # Unpack gravity vector
    ax, ay, az = gravity_vector

    # Model function with known accelerations (ax, ay, az)
    def model(t, x0, v0, a):
        return x0 + v0 * t + 0.5 * a * t**2

    # Fit for initial position (x0) and velocity (v0), using known acceleration (ax, ay, az)
    x_params, _ = curve_fit(lambda t, x0, v0: model(t, x0, v0, ax), t_vals, x_vals, p0=[x_vals[0], x_vals[1] - x_vals[0]])
    y_params, _ = curve_fit(lambda t, y0, v0: model(t, y0, v0, ay), t_vals, y_vals, p0=[y_vals[0], y_vals[1] - y_vals[0]])
    z_params, _ = curve_fit(lambda t, z0, v0: model(t, z0, v0, az), t_vals, z_vals, p0=[z_vals[0], z_vals[1] - z_vals[0]])

    # Return initial positions and velocities, accelerations are known constants
    initial_position = (x_params[0], y_params[0], z_params[0])
    initial_velocity = (x_params[1], y_params[1], z_params[1])

    return {
        'initial_position': initial_position,
        'initial_velocity': initial_velocity,
        'acceleration': (ax, ay, az)
    }


In [ ]:
# Function to compute x, y, z positions with time offset
def x(t, t0, x0, vx0, _ax0):
    return x0 + vx0 * (t - t0) + 0.5 * _ax0 * np.power(t - t0, 2)


# Function to plot predictions
def plot_iteration(idx):
    colors = [
        'red', 'green', 'purple', 'orange', 'yellow',
        'pink', 'brown', 'black', 'gray', 'lightblue', 'lightgreen',
        'lightpink', 'cyan', 'magenta', 'gold', 'lime', 'teal',
        'navy', 'maroon', 'olive', 'coral', 'indigo', 'violet'
    ]

    fig = go.Figure()

    fig.update_layout(
        title=f"Predicted Projectile Motion, Iteration: {idx}",
        scene=dict(
            xaxis_title="x (meters)",
            yaxis_title="y (meters)",
            zaxis_title="z (meters)",
            aspectmode="data"
        ),
        scene_camera=dict(
            up=dict(x=0, y=1, z=0)  # Set y-axis as the vertical axis
        )
    )

    # Points used in prediction
    fig.add_scatter3d(
        x=x_vals[:idx],
        y=y_vals[:idx],
        z=z_vals[:idx],
        mode='markers',
        name='Points Used in Prediction',
        marker=dict(
            size=10,
            color='blue',
            line=dict(color='blue', width=2)
        )
    )

    # Actual trajectory
    fig.add_scatter3d(
        x=x_vals[idx:],
        y=y_vals[idx:],
        z=z_vals[idx:],
        mode='markers',
        name='Actual Trajectory',
        marker=dict(
            size=10,
            color='rgba(0, 0, 0, 0)',
            line=dict(color='green', width=2)
        )
    )

    # Predicted trajectory
    # _t0, _x0, _y0, _z0, _vx0, _vy0, _vz0 = guesses[idx]
    _t0 = 0
    start = None
    end = idx + 1
    params = estimate_trajectory(x_vals[start:end], y_vals[start:end], z_vals[start:end], t_vals[start:end], gravity_vector)
    _x0, _y0, _z0 = params['initial_position']
    _vx0, _vy0, _vz0 = params['initial_velocity']
    _ax0, _ay0, _az0 = params['acceleration']


    fig.add_scatter3d(
        x=x(np.array(t_vals[idx:]), _t0, _x0, _vx0, _ax0),
        y=x(np.array(t_vals[idx:]), _t0, _y0, _vy0, _ay0),
        z=x(np.array(t_vals[idx:]), _t0, _z0, _vz0, _az0),
        mode='markers',
        name='Predicted Trajectory',
        marker=dict(
            size=10,
            color='red',
            line=dict(color='red', width=2)
        )
    )

    xyz_pred = [x(t_vals[-1], _t0, _x0, _vx0, _ax0), x(t_vals[-1], _t0, _y0, _vy0, _ay0), x(t_vals[-1], _t0, _z0, _vz0, _az0)]
    xyz_actual = [x_vals[-1], y_vals[-1], z_vals[-1]]
    error = [abs(a - p) for a, p in zip(xyz_actual, xyz_pred)]
    total_error = np.linalg.norm(error)
    print(f"Absolute error in final prediction vs final trajectory point: {total_error:.2f}m")

    # Show the plot
    fig.show()

In [ ]:
def estimate_gravity_vector_from_throw(x_vals, y_vals, z_vals, t_vals):
    # Combine position data
    positions = np.column_stack((x_vals, y_vals, z_vals))

    # Define the trajectory model with gravity
    def trajectory_model(t, x0, y0, z0, vx0, vy0, vz0, gx, gy, gz):
        positions = np.zeros((len(t), 3))
        positions[:, 0] = x0 + vx0 * t + 0.5 * gx * t**2
        positions[:, 1] = y0 + vy0 * t + 0.5 * gy * t**2
        positions[:, 2] = z0 + vz0 * t + 0.5 * gz * t**2
        return positions.flatten()

    # Initial parameter guess
    # Estimate initial velocity using first few points
    if len(t_vals) >= 2:
        v0_guess = (positions[1] - positions[0]) / (t_vals[1] - t_vals[0])
    else:
        v0_guess = np.zeros(3)

    # Initial parameter guess
    p0 = [
        positions[0, 0],  # x0
        positions[0, 1],  # y0
        positions[0, 2],  # z0
        v0_guess[0],      # vx0
        v0_guess[1],      # vy0
        v0_guess[2],      # vz0
        0.0, 9.8, 0.0    # initial gravity guess (typical Earth gravity in y direction)
    ]

    # Fit the model to the data
    t_array = np.array(t_vals)
    pos_array = positions.flatten()

    # Parameter bounds (broad ranges)
    bounds = ([
        -np.inf, -np.inf, -np.inf,  # position bounds
        -np.inf, -np.inf, -np.inf,  # velocity bounds
        -20, -20, -20               # gravity bounds lower
    ], [
        np.inf, np.inf, np.inf,     # position bounds
        np.inf, np.inf, np.inf,     # velocity bounds
        20, 20, 20                  # gravity bounds upper
    ])

    params, _ = curve_fit(
        trajectory_model,
        t_array,
        pos_array,
        p0=p0,
        bounds=bounds,
        method='trf'
    )

    # Extract gravity vector
    gravity_vector = params[6:9]

    return gravity_vector

In [ ]:
x,y,z,t = load_trajectory_data("test_prediction_10.json")
gravity_vector = estimate_gravity_vector_from_throw(x, y, z, t)
print(f"Estimated Gravity Vector: {gravity_vector}")

Estimated Gravity Vector: [-0.5096488  10.54434098  2.40276155]
